In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
data = pd.read_pickle("../src/original_data/covid_data_2020_03.pickle")
data

In [ ]:
#list(data.columns)

In [ ]:
Q3 = data[['avoid_busy_places','avoid_public_places','maintain_distance','adjust_school_work','quarantine_symptoms', 'quarantine_no_symptoms','no_avoidance_behaviors']]
Q3

In [ ]:
Q3_none = Q3.query('no_avoidance_behaviors==1')
Q3_none

In [ ]:
Q3.insert(7,"sum", 0)
Q3.loc[Q3['no_avoidance_behaviors']!=1, 'sum'] = Q3.loc[Q3['no_avoidance_behaviors']!=1,
   ['avoid_busy_places','avoid_public_places','maintain_distance',
    'adjust_school_work','quarantine_symptoms', 'quarantine_no_symptoms']].sum(axis='columns')

In [ ]:
Q3

In [ ]:
background = pd.read_pickle("../src/original_data/background_data_merged.pickle")
#print(list(background.columns))
background

In [ ]:
#list(background.columns)

In [ ]:
select_background = background.loc[:, ["age","age_group","female","education_cbs","hh_members","hh_children","dom_situation","location_urban"]].dropna(axis=0, how='any')
select_background

In [ ]:
merge_data = Q3.join(select_background, on="personal_id", how="inner")
merge_data

In [ ]:
print(dict(enumerate(merge_data['age_group'].cat.categories)))
dict(enumerate(merge_data['education_cbs'].cat.categories))

In [ ]:
dict(enumerate(merge_data['dom_situation'].cat.categories))

In [ ]:
dict(enumerate(merge_data['location_urban'].cat.categories))

In [ ]:
merge_data['edu_index'] = merge_data['education_cbs'].cat.codes.replace({7:1,0:2,1:3,2:4,3:5,4:6,5:7,6:np.nan})
merge_data['age_index'] = merge_data['age_group'].cat.codes.replace({0:1, 1:2, 2:3})
merge_data['dom_situation_index'] = merge_data['dom_situation'].cat.codes.replace({2:1,3:1,4:1})
merge_data['hh_members_dummy'] = (merge_data['hh_members']>1).astype(int)
merge_data['hh_children_dummy'] = (merge_data['hh_children']>0).astype(int)
merge_data['location_urban_index'] = 5 - merge_data['location_urban'].cat.codes
merge_data['age'] = merge_data['age'].astype('int')
merge_data.head(20)

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression

# y = merge_data.loc[:, "sum"]
# x = merge_data.loc[:, ['age_code','edu_code','female']]

# # y = y[:, np.newaxis]

# model = LinearRegression()
# model.fit(x, y)
# predicts = model.predict(x)
# R2 = model.score(x, y)
# print('R2 = %.3f' % R2)
# coef = model.coef_
# intercept = model.intercept_ 
# print(model.coef_, model.intercept_)

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

y = merge_data.loc[:, "sum"]
x = merge_data.loc[:, ['age_dummy','edu_dummy','female']]
x = sm.add_constant(x) # add constant if need intercept
model = sm.OLS(y, x).fit() # ols fit
print(model.summary()) # print fit result

# plot
# predicts = model.predict() # predict y_hat
# x = merge_data.loc[:, ['age_code']]
# plt.scatter(x, y, label='y true') # point plot
# plt.plot(x, predicts, color = 'red', label='y hat')
# plt.legend()
# plt.show()

In [ ]:

y = merge_data.loc[:, "sum"]
x = merge_data.loc[:, ['age','edu_index','female','dom_situation_index','hh_members_dummy','hh_children_dummy','location_urban_index']]
x.insert(1,'age_square', merge_data.loc[:,'age'].pow(2))
x.insert(2,'age_cube', merge_data.loc[:, 'age'].pow(3))

x = sm.add_constant(x) # add constant if need intercept
model = sm.OLS(y, x).fit() # ols fit
print(model.summary()) # print fit result

In [ ]:

y = merge_data.loc[:, "sum"]
x = merge_data.loc[:, ['age_index','edu_index','female','dom_situation_index','hh_members_dummy','hh_children_dummy','location_urban_index']]
x.insert(1,'age_index_square', merge_data.loc[:,'age_index'].pow(2))
x.insert(2,'age_index_cube', merge_data.loc[:, 'age_index'].pow(3))

x = sm.add_constant(x) # add constant if need intercept
model = sm.OLS(y, x).fit() # ols fit
print(model.summary()) # print fit result